In [0]:
%pip install databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
%run "../Configuraciones/variables_configuraciones"

In [0]:
print(RUTA_CARPETA_GIT)
print(NOMBRE_JOB_LEVANTAR_AMBIENTE)

In [0]:
job_config =    {
    "email_notifications": {
        "no_alert_for_skipped_runs": False
    },
    "format": "MULTI_TASK",
    "max_concurrent_runs": 1,
    "name": NOMBRE_JOB_LEVANTAR_AMBIENTE,
    "performance_target": "PERFORMANCE_OPTIMIZED",
    "queue": {
        "enabled": True
    },
    "tasks": [
        {
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/00_Borrar_todo",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "Borrar_todo",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "Borrar_todo"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/01_creacion_tablas",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "creacion_tablas",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "creacion_tablas"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/02_insertar_valores_casteo_columnas",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "poblar_tabla_casteo_columnas",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "poblar_tabla_casteo_columnas"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/03_insertar_valores_mapeo_cabeceras",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "poblar_tabla_mapeo_cabaceras",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "poblar_tabla_mapeo_cabaceras"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/04_insertar_valores_catalogos",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "poblar_catalogos",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "poblar_catalogos"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/05_insertar_valores_hmg",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "poblar_homologadores",
            "timeout_seconds": 0,
            "webhook_notifications": {}
        },
        {
            "depends_on": [
                {
                    "task_key": "poblar_homologadores"
                }
            ],
            "email_notifications": {},
            "notebook_task": {
                "notebook_path": f"{RUTA_CARPETA_GIT}/Databricks/LevantarAmbiente/06_insertar_valores_dim",
                "source": "WORKSPACE"
            },
            "run_if": "ALL_SUCCESS",
            "task_key": "poblar_dimension",
            "timeout_seconds": 0
        }
    ],
    "timeout_seconds": 0,
    "webhook_notifications": {}
}

In [0]:
from databricks.sdk.service.jobs import JobSettings as Job
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
new_config = Job.from_dict(job_config)
new_job = w.jobs.create(**new_config.as_shallow_dict())

### JobId:

In [0]:
print("Nuevo job creado con ID:", new_job.job_id)

In [0]:
run = w.jobs.run_now(job_id=new_job.job_id)
print("Run ID:", run.run_id)